# Multi Layer Perceptron (MLP)

> Simple feedforward Multilayer perceptron model

In [1]:
#| default_exp models.mlp

In [2]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

In [3]:
#| export
import torch.nn as nn
import torch
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST


from pytorch_lightning import LightningModule, Trainer
from torchmetrics import Accuracy
from hydra.utils import instantiate
from omegaconf import OmegaConf
from matplotlib import pyplot as plt

from nimrod.utils import get_device
from nimrod.image.datasets import ImageDataset, MNISTDataModule

# from IPython.core.debugger import set_trace

In [4]:
show_doc(MNISTDataModule)

---

[source](https://github.com/slegroux/nimrod/blob/main/nimrod/image/datasets.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### MNISTDataModule

>      MNISTDataModule (data_dir:str='~/Data/',
>                       train_val_test_split:List[float]=[0.8, 0.1, 0.1],
>                       batch_size:int=64, num_workers:int=0,
>                       pin_memory:bool=False, persistent_workers:bool=False)

*A DataModule standardizes the training, val, test splits, data preparation and transforms. The main advantage is
consistent data splits, data preparation and transforms across models.

Example::

    import lightning.pytorch as L
    import torch.utils.data as data
    from pytorch_lightning.demos.boring_classes import RandomDataset

    class MyDataModule(L.LightningDataModule):
        def prepare_data(self):
            # download, IO, etc. Useful with shared filesystems
            # only called on 1 GPU/TPU in distributed
            ...

        def setup(self, stage):
            # make assignments here (val/train/test split)
            # called on every process in DDP
            dataset = RandomDataset(1, 100)
            self.train, self.val, self.test = data.random_split(
                dataset, [80, 10, 10], generator=torch.Generator().manual_seed(42)
            )

        def train_dataloader(self):
            return data.DataLoader(self.train)

        def val_dataloader(self):
            return data.DataLoader(self.val)

        def test_dataloader(self):
            return data.DataLoader(self.test)

        def on_exception(self, exception):
            # clean up state after the trainer faced an exception
            ...

        def teardown(self):
            # clean up state after the trainer stops, delete files...
            # called on every process in DDP
            ...*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| data_dir | str | ~/Data/ | path to source data dir |
| train_val_test_split | List | [0.8, 0.1, 0.1] | train val test % |
| batch_size | int | 64 | size of compute batch |
| num_workers | int | 0 | num_workers equal 0 means that it’s the main process that will do the data loading when needed, num_workers equal 1 is the same as any n, but you’ll only have a single worker, so it might be slow |
| pin_memory | bool | False | If you load your samples in the Dataset on CPU and would like to push it during training to the GPU, you can speed up the host to device transfer by enabling pin_memory. This lets your DataLoader allocate the samples in page-locked memory, which speeds-up the transfer |
| persistent_workers | bool | False |  |

## Basic model

In [53]:
#| export
class MLP(nn.Module):
    def __init__(
                self,
                n_in:int, # input dimension e.g. (H,W) for image
                n_h:int, # hidden dimension
                n_out:int, # output dimension (= number of classes for classification)
                dropout:float=0.2
                ) -> None:
        super().__init__()
        l1 = nn.Linear(n_in, n_h)
        l2 = nn.Linear(n_h, n_out)
        relu = nn.ReLU()
        dropout = nn.Dropout(dropout)
        self.layers = nn.Sequential(l1, dropout, relu, l2)
        
    def forward(self, x: torch.Tensor # dim (B, H*W)
                ) -> torch.Tensor:
        return self.layers(x)

### Usage

In [54]:
image = torch.rand((5, 28*28))
mlp = MLP(n_in=28*28, n_h=64, n_out=10)
out = mlp(image)
print(out.shape)

torch.Size([5, 10])


### Basic training
#### Data Module
Data module
c.f. recipes/image/mnist

```bash
cat ../config/image/data/mnist.yaml
```

In [7]:
# load from config file
cfg = OmegaConf.load('../config/image/data/mnist.yaml')
print(cfg.datamodule)
datamodule = instantiate(cfg.datamodule)
datamodule.prepare_data()
datamodule.setup()
x = datamodule.data_test[0][0] # (C, H, W)
print(len(datamodule.data_test))
label = datamodule.data_test[0][1] #(int)
print("original shape (C,H,W): ", x.shape)
print("reshape (C,HxW): ", x.view(x.size(0), -1).shape)
print(x[0][1])

{'_target_': 'nimrod.image.datasets.MNISTDataModule', 'data_dir': '../data/image', 'train_val_test_split': [0.8, 0.1, 0.1], 'batch_size': 64, 'num_workers': 0, 'pin_memory': False, 'persistent_workers': False}
7000
original shape (C,H,W):  torch.Size([1, 28, 28])
reshape (C,HxW):  torch.Size([1, 784])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])


In [8]:
# using default Pytorch datasets
train_dataset = MNIST("../data/image", train=True, download=True, transform=ToTensor())
test_dataset = MNIST("../data/image", train=False, download=True, transform=ToTensor())

# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# using nimrod datamodule
train_loader = datamodule.train_dataloader()
val_loader = datamodule.val_dataloader()
test_loader = datamodule.test_dataloader()

In [9]:
type(datamodule.data_test)

torch.utils.data.dataset.Subset

#### Hardware acceleration

In [10]:
# device = "mps" if torch.backends.mps.is_available() else "cpu"
device = "cpu" # for CI on cpu instance
device = torch.device(device)
model = mlp.to(device)

#### Loss & optimizer setup

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

#### Training loop

In [12]:

%%time
n_epochs = 1
for epoch in range(n_epochs):
    model.train()
    for images, labels in train_loader:
        images = images.view(-1, 28*28)
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            # model expects input (B,H*W)
            images = images.view(-1, 28*28).to(device)
            images = images.to(device)
            labels = labels.to(device)
            # Pass the input through the model
            outputs = model(images)
            # Get the predicted labels
            _, predicted = torch.max(outputs.data, 1)

            # Update the total and correct counts
            total += labels.size(0)
            correct += (predicted == labels).sum()

        # Print the accuracy
        print(f"Epoch {epoch + 1}: Accuracy = {100 * correct / total:.2f}%")


Epoch 1: Accuracy = 73.59%
CPU times: user 2.55 s, sys: 739 ms, total: 3.28 s
Wall time: 2.7 s


## Integrated model + training settings

In [ ]:
#| export
class MLP_PL(MLP, LightningModule):
    def __init__(self,
                n_in:int, # input dimension e.g. (H,W) for image
                n_h:int, # hidden dimension
                n_out:int, # output dimension (= number of classes for classification)
                dropout:float=0.2, # dropout factor
                lr:float=1e-3 # learning rate
                ):

        super(MLP_PL, self).__init__(n_in, n_h, n_out, dropout)

        self.save_hyperparameters()
        self.loss = nn.CrossEntropyLoss()
        self.accuracy = Accuracy(task="multiclass", num_classes=10)
        self.lr = lr

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer
    
    def _step(self, batch, batch_idx):
        x, y = batch
        x = x.view(x.size(0), -1)
        y_hat = self.forward(x)
        loss = self.loss(y_hat, y)
        acc = self.accuracy(y_hat, y)
        return loss, acc

    def training_step(self, batch, batch_idx):
        loss, acc = self._step(batch, batch_idx)
        metrics = {"train/loss": loss, "train/acc": acc}
        self.log_dict(metrics, on_epoch=True)
        return loss
    
    def validation_step(self, batch, batch_idx, prog_bar=True, on_step=False, on_epoch=True, sync_dist=True):
        loss, acc = self._step(batch, batch_idx)
        metrics = {"val/loss":loss, "val/acc": acc}
        self.log_dict(metrics, on_step=on_step, on_epoch=on_epoch, sync_dist=sync_dist)
    
    def test_step(self, batch, batch_idx, prog_bar=True, on_step=False, on_epoch=True, sync_dist=True):
        loss, acc = self._step(batch, batch_idx)
        metrics = {"test/loss":loss, "test/acc": acc}
        self.log_dict(metrics, on_step=on_step, on_epoch=on_epoch, sync_dist=sync_dist)

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        x, y = batch
        x = x.view(x.size(0), -1)
        y_hat = self.forward(x)
        return y_hat.argmax(dim=1)


### Usage

In [73]:
MLP_PL.mro()

[__main__.MLP_PL,
 __main__.MLP,
 pytorch_lightning.core.module.LightningModule,
 lightning_fabric.utilities.device_dtype_mixin._DeviceDtypeModuleMixin,
 pytorch_lightning.core.mixins.hparams_mixin.HyperparametersMixin,
 pytorch_lightning.core.hooks.ModelHooks,
 pytorch_lightning.core.hooks.DataHooks,
 pytorch_lightning.core.hooks.CheckpointHooks,
 torch.nn.modules.module.Module,
 object]

In [82]:
# wrap simple model in modularized model
mlp_pl = MLP_PL(28*28, 64, n_out=10, dropout=0.2, lr=1e-3)

# fake input
b = torch.rand((5,1, 28*28))

# move model and data to hardware
model = mlp_pl.to(device)

b = b.to(device)
y_hat = mlp_pl(b)
print(y_hat.shape)

# real data
batch = next(iter(test_loader))
print(batch[0].shape, batch[1].shape)
print(model.predict_step(batch, 0))

torch.Size([5, 1, 10])
torch.Size([64, 1, 28, 28]) torch.Size([64])
tensor([1, 3, 3, 7, 3, 0, 5, 3, 7, 3, 3, 7, 1, 9, 3, 3, 3, 5, 3, 3, 1, 9, 1, 3,
        7, 7, 1, 1, 3, 3, 1, 1, 5, 1, 7, 3, 1, 3, 3, 3, 0, 3, 3, 3, 3, 3, 9, 3,
        1, 9, 1, 9, 3, 9, 3, 1, 3, 5, 3, 3, 7, 3, 3, 3])


In [ ]:
print(model.lr)

0.001


In [ ]:
# print(bb)

## Integrated trainer

```python
trainer = Trainer(accelerator='mps', devices = 1, max_epochs=1)
trainer.fit(mlp_pl, datamodule.data_train)
trainer.fit(mlp_pl, datamodule.data_train)
```

## Training scripts with config file 

To check an example script leveraging model training with configurable yaml files check recipes folder

```bash
cd recipes/image/mnist
python train.py trainer.max_epochs 20 trainer.accelerator='mps' datamodule.num_workers=0
```

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()